In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import CuDNNGRULast as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs_sw', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'cudnngrulast_test')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'cudnngrulast_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)
RESULT_CSV = os.path.join(RESULTS_PATH, 'cudnngrulast_sw.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 168s 2ms/step - loss: 0.5417 - acc: 0.8184 - val_loss: 0.2652 - val_acc: 0.9346


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.45764, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 2/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.2443 - acc: 0.9329 - val_loss: 0.2029 - val_acc: 0.9447

Epoch 00002: _f1_monitor improved from 0.45764 to 0.58620, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 3/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.2155 - acc: 0.9391 - val_loss: 0.1901 - val_acc: 0.9458

Epoch 00003: _f1_monitor improved from 0.58620 to 0.60451, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 4/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.2052 - acc: 0.9420 - val_loss: 0.1887 - val_acc: 0.9472

Epoch 00004: _f1_monitor

105000/105000 [==============================] - 165s 2ms/step - loss: 0.2922 - acc: 0.9091 - val_loss: 0.2799 - val_acc: 0.9146

Epoch 00004: _f1_monitor improved from 0.61985 to 0.63591, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 5/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.2818 - acc: 0.9122 - val_loss: 0.2707 - val_acc: 0.9158

Epoch 00005: _f1_monitor improved from 0.63591 to 0.64653, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 6/100
105000/105000 [==============================] - 165s 2ms/step - loss: 0.2765 - acc: 0.9133 - val_loss: 0.2694 - val_acc: 0.9163

Epoch 00006: _f1_monitor did not improve from 0.64653
Epoch 7/100
105000/105000 [==============================] - 165s 2ms/step - loss: 0.2728 - acc: 0.9141 - val_loss: 0.2734 - val_acc: 0.9142

Epoch 00007: _f1_monit

Epoch 6/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.4037 - acc: 0.8535 - val_loss: 0.3925 - val_acc: 0.8547

Epoch 00006: _f1_monitor did not improve from 0.77528
Epoch 7/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.3981 - acc: 0.8549 - val_loss: 0.3894 - val_acc: 0.8545

Epoch 00007: _f1_monitor improved from 0.77528 to 0.78114, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_4.h5
Epoch 8/100
105000/105000 [==============================] - 167s 2ms/step - loss: 0.3901 - acc: 0.8594 - val_loss: 0.3917 - val_acc: 0.8557

Epoch 00008: _f1_monitor did not improve from 0.78114
Epoch 9/100
105000/105000 [==============================] - 167s 2ms/step - loss: 0.3855 - acc: 0.8610 - val_loss: 0.4015 - val_acc: 0.8548

Epoch 00009: _f1_monitor did not improve from 0.78114
Epoch 10/100
105000/105000 [==============================] - 167s 2ms/step - loss: 0.

...


Epoch 00014: _f1_monitor did not improve from 0.73796
Epoch 15/100
105000/105000 [==============================] - 165s 2ms/step - loss: 0.3419 - acc: 0.8804 - val_loss: 0.3900 - val_acc: 0.8641

Epoch 00015: _f1_monitor improved from 0.73796 to 0.73822, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_11.h5

Epoch 00015: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 11th grain: f1_score - 0.7382186959587047, acc - 0.8641333333333333
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 167s 2ms/step - loss: 0.9794 - acc: 0.6279 - val_loss: 0.5939 - val_acc: 0.7833

Epoch 00001: _f1_monitor improved from -inf to 0.58822, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_12.h5
Epoch 2/100
105000/105000 [==============================] - 165s 2ms/step - loss: 0.582

105000/105000 [==============================] - 165s 2ms/step - loss: 0.3670 - acc: 0.8716 - val_loss: 0.3796 - val_acc: 0.8712

Epoch 00013: _f1_monitor improved from 0.73319 to 0.73434, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_13.h5
Epoch 14/100
105000/105000 [==============================] - 165s 2ms/step - loss: 0.3656 - acc: 0.8728 - val_loss: 0.3832 - val_acc: 0.8698

Epoch 00014: _f1_monitor did not improve from 0.73434
Epoch 15/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.3614 - acc: 0.8740 - val_loss: 0.3815 - val_acc: 0.8710

Epoch 00015: _f1_monitor did not improve from 0.73434
Epoch 16/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.3619 - acc: 0.8745 - val_loss: 0.3833 - val_acc: 0.8708

Epoch 00016: _f1_monitor improved from 0.73434 to 0.73454, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/

Epoch 8/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.4936 - acc: 0.7964 - val_loss: 0.5192 - val_acc: 0.7805

Epoch 00008: _f1_monitor improved from 0.70909 to 0.71046, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_15.h5
Epoch 9/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.4891 - acc: 0.7982 - val_loss: 0.5275 - val_acc: 0.7782

Epoch 00009: _f1_monitor improved from 0.71046 to 0.71087, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_15.h5
Epoch 10/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.4876 - acc: 0.7999 - val_loss: 0.5267 - val_acc: 0.7767

Epoch 00010: _f1_monitor did not improve from 0.71087
Epoch 11/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.4839 - acc: 0.8008 - val_loss: 0.5206 - val_acc: 0.7806

Epoch 

Epoch 7/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.3101 - acc: 0.9037 - val_loss: 0.3075 - val_acc: 0.9043

Epoch 00007: _f1_monitor did not improve from 0.72577
Epoch 8/100
105000/105000 [==============================] - 166s 2ms/step - loss: 0.3055 - acc: 0.9057 - val_loss: 0.3146 - val_acc: 0.9044

Epoch 00008: _f1_monitor improved from 0.72577 to 0.72995, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_17.h5
Epoch 9/100
105000/105000 [==============================] - 167s 2ms/step - loss: 0.3047 - acc: 0.9059 - val_loss: 0.3080 - val_acc: 0.9041

Epoch 00009: _f1_monitor did not improve from 0.72995
Epoch 10/100
105000/105000 [==============================] - 167s 2ms/step - loss: 0.3016 - acc: 0.9071 - val_loss: 0.3076 - val_acc: 0.9025

Epoch 00010: _f1_monitor did not improve from 0.72995

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Ep

105000/105000 [==============================] - 165s 2ms/step - loss: 0.3341 - acc: 0.8799 - val_loss: 0.3537 - val_acc: 0.8710

Epoch 00012: _f1_monitor did not improve from 0.69828

Epoch 00012: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 19th grain: f1_score - 0.6982817264300593, acc - 0.8711333333333333


In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6310752692961574
The 1th f1: 0.511014811268255
The 2th f1: 0.6748646976986535
The 3th f1: 0.6337691527685467
The 4th f1: 0.7837930982395139
The 5th f1: 0.7291057007084671
The 6th f1: 0.724902957329557
The 7th f1: 0.7581070892773799
The 8th f1: 0.6930167473861454
The 9th f1: 0.6621084544772902
The 10th f1: 0.7014704954017317
The 11th f1: 0.7382186959587047
The 12th f1: 0.732359573146065
The 13th f1: 0.7345426421813839
The 14th f1: 0.68970497898588
The 15th f1: 0.712222545082245
The 16th f1: 0.5558389298738536
The 17th f1: 0.7324725422424961
The 18th f1: 0.5693697790187477
The 19th f1: 0.6982817264300593
The average f1 of val is 0.6833119943385566


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)

In [12]:
print("Hello world")

Hello world
